# HAPI V2 Training

## Chapter 2 - MDS with GetData

This sample shows how to create a new MDS request.

#### Table of Content
- Section 1 - Initialization
- Section 2 - Universes/FieldLists/Trigger
- Section 3 - GetData Request
- Section 4 - Download dataset

### Section 1 - Initialization
- Import related libraries
- Mention the requirements.txt (e.g. pyjwt need to be installed)

In [ ]:
import json
import requests
from datetime import datetime

- Use the beap_auth.py to handle HAPI authentication
- Load credentials from *credential.txt* file you have obtained from *https://console.bloomberg.com*

In [ ]:
from beap_auth import Credentials, BEAPAdapter
CREDS = Credentials.from_file('Credentials/credential_MDS.txt')

adapter = BEAPAdapter(CREDS, api_version='2') # if not specified, api_version will default to '2' except for CUSTs on exception list
session = requests.Session()
session.mount('https://', adapter)
HOST = 'https://api.bloomberg.com' # 'https://api.blpprofessional.com' for China

- Construct the URL that will be the prefix for the other requests
Catalogs

https://service.blpprofessional.com/track_download/assets/HAPI/#tag/catalog

https://service.blpprofessional.com/track_download/assets/data-license/#2-5-catalogs

In [ ]:
account_url = HOST+'/eap/catalogs/781890/'
bbg_url = HOST+'/eap/catalogs/bbg/'
print("Account catalog URL  :", account_url)
print("Bloomberg catalog URL:", bbg_url)

- Create a variable using your own login name, which will be used as identifier later

In [ ]:
login_name = 'replace_this_with_your_login' # e.g. yliu1436

## Section 2 - Three Resources
#### CRUD actions on resources:
- C: Resources can be created using **POST**.
- R: Resources can be read using **GET**.
- U: Resources can be updated using **PATCH**.
- D: Resources **CANNOT** be deleted

### Section 2.1 Query Universes

- We can use a pre-defined/dynamic universe provided in the 'bbg' catalog.  This universe is only expanded when the request is being transformed to an output when your request is scheduled to run.
BUT,
- For the purpose of this demo we will be using specific TICKERS
- https://service.blpprofessional.com/track_download/assets/HAPI/#tag/universes

#### 2.1.1 Query BBG predefined universes

In [ ]:
url = bbg_url + 'universes/?pageSize=1500'
print("GET URL:", url)
resp = session.get(url)
print('RESPONSE:\n', json.dumps(resp.json(), indent=4))

#### 2.1.2 - Query custom universes
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/getUniverses

In [ ]:
url = account_url + 'universes/?pageSize=1500'
print("GET URL:", url)
resp = session.get(url)
print('Total Custom Universes:', resp.json()['totalItems'])
print('Total Pages:', resp.json()['pageCount'])
print('RESPONSE:\n', json.dumps(resp.json(), indent=4))

<div class="alert alert-block alert-success">
    
## Task #1

- Find the total number of BBG pre-defined universes.

</div>

In [ ]:
# type your code here

#### 2.1.3 Create Custom Universe
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/postUniverse

In [ ]:

universe_id = 'HAPI2TrainingUniverse'+ login_name
universe_payload = {
    '@type': 'Universe',
    'identifier': universe_id,
    'title': 'HAPI V2 Training - Universe of ' + login_name,
    'description': 'Some description',
    'contains': [
        {
            '@type': 'Identifier',
            'identifierType': 'BB_GLOBAL',
            'identifierValue': 'BBG009S3NB30',  # GOOG US Equity
        },
        {
            '@type': 'Identifier',
            'identifierType': 'ISIN',
            'identifierValue': 'US88160R1014',  # TSLA US Equity
        },
    ]
}


url = account_url+'universes/'
print(datetime.now(), "POST URL:", url)
print('POST DATA:\n',json.dumps(universe_payload, indent=4))
# Note the universes URL we are POSTing to is same as where we GET from in previous step
response = session.post(url, json=universe_payload)

# Check it went well and extract the URL of the created universe
print('RESPONSE:\n', json.dumps(response.json(), indent=4))
response.raise_for_status()

universe_location = response.headers['Location']
universe_url = HOST+universe_location
print(datetime.now(), 'Universe successfully created at', universe_url)

#### 2.1.4 Query the list of securities in the newly created universe

In [ ]:
resp = session.get(universe_url)
resp.json()['contains']

#### 2.1.5 Add a new security to the universe.
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/patchUniverse

In [ ]:
update_payload = {
    'contains': universe_payload['contains'] + [{'@type':'Identifier', 'identifierType': 'TICKER', 'identifierValue' : '5 HK Equity'}]
}

# Use HTTP PATCH to update the resource
resp = session.patch(universe_url, json=update_payload)
resp

In [ ]:
resp = session.get(universe_url)
resp.json()['contains']

#### 2.1.6 Delete a security from a universe
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/patchUniverse

In [ ]:
update_payload['contains'].remove({'@type': 'Identifier','identifierType': 'BB_GLOBAL','identifierValue': 'BBG009S3NB30'})

update_payload_v2 = {
    'contains': update_payload['contains']
}

print(datetime.now(), 'POST URL:', universe_url)
print(datetime.now(), 'POST DATA:', update_payload_v2)

# Use HTTP PATCH to update the resource
resp = session.patch(universe_url, json=update_payload_v2)
resp

In [ ]:
resp = session.get(universe_url)
resp.json()['contains']

### Section 2.2 Field List

- Create the field list component. Please note that "@type" is different for fields used GETDATA and GETHISTORY.
- GETDATA will have '@type': 'DataFieldList'
- GETHISTORY will have '@type': 'HistoryFieldList'
- https://service.blpprofessional.com/track_download/assets/HAPI/#tag/fieldLists

#### 2.2.1 - Query Field Lists
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/getFieldLists

In [ ]:
url = account_url + 'fieldLists/'
print("GET URL:", url)
resp = session.get(url)
print('Total field lists:', resp.json()['totalItems'])
print('Total Pages:', resp.json()['pageCount'])
print('RESPONSE:\n', json.dumps(resp.json(), indent=4))

- You can specify a larger pageSize to avoid paging

In [ ]:
url = account_url + 'fieldLists/?pageSize=1500'
print("GET URL:", url)
resp = session.get(url)
print('Total field lists:', resp.json()['totalItems'])
print('Total Pages:', resp.json()['pageCount'])
print('RESPONSE:\n', json.dumps(resp.json(), indent=4))

#### 2.2.2 Create a DataFieldList
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/postFieldList

In [ ]:
fieldlist_id = 'HAPI2TrainingDataFieldList'+login_name
fieldlist_payload = {
    '@type': 'DataFieldList',
    'identifier': fieldlist_id,
    'title': 'HAPI v2 Training - DataFieldList of '+login_name,
    'description': 'Some description',
    'contains': [
        {'cleanName': 'name'},
        {'cleanName': 'pxLast'}
    ],
}
fieldlist_payload
url = account_url+'fieldLists/'
print(datetime.now(), 'POST URL:', url)
print(datetime.now(), 'POST DATA:\n', json.dumps(fieldlist_payload, indent=4))
resp = session.post(url, json=fieldlist_payload)
# Check it went well and extract the URL of the created field list
print('RESPONSE: \n', json.dumps(resp.json(), indent = 4))
resp.raise_for_status()

fieldlist_location = resp.headers['Location']
fieldlist_url = HOST+fieldlist_location
print(datetime.now(), 'Field list successfully created:', fieldlist_url)

#### 2.2.3 Inspect the newly-created field list component

In [ ]:
resp=session.get(fieldlist_url)
resp.json()['contains']

#### 2.2.4 Add a field to the field list

In [ ]:
update_payload = {
    'contains': fieldlist_payload['contains'] + [{'cleanName':'pxBid'}, {'cleanName':'pxAsk'}]
}

# Use HTTP PATCH to update the resource
resp = session.patch(fieldlist_url, json=update_payload)
resp

In [ ]:
resp = session.get(fieldlist_url)
resp.json()['contains']

#### 2.2.5 Delete a field from the field lists

In [ ]:
update_payload['contains'].remove({'cleanName':'pxBid'})

update_payload_v2 = {
    'contains': update_payload['contains']
}

print(datetime.now(), 'POST URL:', fieldlist_url)
print(datetime.now(), 'POST DATA:', update_payload)

# Use HTTP PATCH to update the resource
resp = session.patch(fieldlist_url, json=update_payload_v2)
resp

In [ ]:
resp = session.get(fieldlist_url)
resp.json()['contains']

### Section 2.3 Trigger

Trigger defines time and recurrence of request.

https://service.blpprofessional.com/track_download/assets/HAPI/#tag/triggers

### 2.3.1 - Query existing triggers
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/getTriggers

- Again, you can specify a larger pageSize to avoid paging

In [ ]:
url = account_url + 'triggers/?pageSize=1500'
print("GET URL:", url)
resp = session.get(url)
print('Total Custom Triggers:', resp.json()['totalItems'])
print('Total Pages:', resp.json()['pageCount'])
print('RESPONSE: \n', json.dumps(resp.json(), indent = 4))

### 2.3.2 Fetch and print out the pre-defined BBG trigger
We use a pre-defined trigger component. Bloomberg provides a common trigger component named 'submit', which is equivalent to the traditional 'adhoc' program flag in DL per-security. It will run your request as soon as allowable after you submit your request.

In [ ]:
trigger_url = bbg_url+'triggers/submit/'
resp = session.get(trigger_url)
resp.json()

### 2.3.3 Create your own custom trigger
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/postTrigger

In [ ]:
trigger_id = 'HAPI2TrainingTrigger1'+login_name

trigger_payload = {
    '@type': 'ScheduledTrigger',
    'identifier': trigger_id,
    'title': trigger_id,
    'frequency': 'daily',
    'startDate': '2020-05-11',  # YYYY-MM-DD
    'startTime': '18:00:00',    # HH:MM:SS
}

url = account_url+'triggers/'

print('POST URL:', url)
print('POST DATA:\n', json.dumps(trigger_payload, indent=4))
resp = session.post(url, json=trigger_payload)
resp.json()

In [ ]:
session.get(url+trigger_id).json()

#### 2.3.4 Update trigger resource.

In [ ]:
trigger_payload_updated={
    'title': trigger_id,
    'description' : "Weekend Trigger",
    'frequency': 'weekend',
    'startDate': '2020-05-16',  # YYYY-MM-DD
    'startTime': '18:00:00'   # HH:MM:SS
}

# Use HTTP PATCH to update the resource
resp = session.patch(url+trigger_id, json=trigger_payload_updated)
resp

### Summary of Universe, fieldLists and Triggers.

In [ ]:
# universe_url=account_url+'universes/HAPI2TrainingUniverseyliu1436'
# fieldlist_url = account_url+'fieldLists/HAPI2TrainingHistoryFieldListyliu1436'
# Verify whether your have created correct resources
print("Universe you selected is "+universe_url)
print("FieldLists you selected is "+fieldlist_url)
print("Trigger you selected is "+trigger_url)

## Section 3 - GetData Request

If we've got this far, we should have all the components required for defining a request

### 3.1 Create the request component
https://service.blpprofessional.com/track_download/assets/HAPI/#operation/postRequest

In [ ]:
# NOTE The request_id must be 21 characters or less and should be Unique as well.
# Therefore, please increment the request_id each time
request_id = login_name+'GetData001' # increment the number 001 when necessary
request_payload = {
    '@type': 'DataRequest',
    'identifier': request_id,
    'title': request_id,
    'description': 'Get Data Request for ' + login_name,
    # 3 key resources
    'universe': universe_url,
    'fieldList': fieldlist_url,
    'trigger': trigger_url,
    # optional formatting
    'formatting': {
        '@type': 'DataFormat',
        'columnHeader': True,
        'dateFormat': 'yyyymmdd',
        'delimiter': '|',
        'fileType': 'unixFileType',
        'outputFormat': 'variableOutputFormat',
    },
    'pricingSourceOptions': {
        '@type': 'DataPricingSourceOptions',
        'prefer': {'mnemonic': 'BGN'}
    },
}


url = account_url+'requests'
print(datetime.now(), 'POST URL:', url)
print(datetime.now(), 'POST DATA:\n', json.dumps(request_payload, indent=4))

resp = session.post(url, json=request_payload)
# Check it went well and extract the URL of the created request
print(resp.json())
resp.raise_for_status()

request_location = resp.headers['Location']
request_url = HOST+request_location
print(datetime.now(), 'Request has been successfully created:',request_url)

### Section 4 - Download Custom Dataset

Once request was successfully created it is accepted to execution by the HAPI service.  
Multiple ways in which clients could download the response dataset.
-  Continuously polling the URL of the dataset to see if the snapshot is available for download, if yes then use the download_distribution() or any custom method to download the dataset.
-  Use SSE notification which will return an event to notify that the snapshot is now available for download.

In [ ]:
def download_distribution(session_, url, out_path, chunk_size=8192,
                          stream=True, headers=None):
    """
    Function to download the data to an output directory

    This function opts for the gzip output encoding by default and allows the
    user to specify the output location of this download. This function works
    for a single endpoint.

    You may set the 'Accept-Encoding' header to 'Identity' if you do not
    want to receive the gzipped file.

    Set 'chunk_size' to a larger byte size to speed up download process on
    larger downloads.
    """
    headers = headers or {'Accept-Encoding': 'Identity'}

    if headers['Accept-Encoding'] == 'gzip':
        out_path += '.gz'

    print(datetime.now(), 'Start downloading:', url)
    with session_.get(url, stream=stream, headers=headers) as response_:
        with open(out_path, 'wb') as out_file:
            for chunk in response_.raw.stream(chunk_size, decode_content=False):
                out_file.write(chunk)
            print(datetime.now(), 'Downloaded to:', out_file.name)
            return response_

### 4.1 Poll until distriubtion is ready for download
- The way you poll is keep running the below cell and see if there is any date within key "contains" is returned. If yes, then the response file is available for download for that date. If nothing is returned, then the file is still not available.

In [ ]:
snap_url = '{a}datasets/{r}/snapshots/'.format(a=account_url, r =request_id  )

snapshot_resp = session.get(snap_url)
resp = snapshot_resp.json()
resp

### 4.2 Download distribution

Now that we see a date, we can go ahead and create the URL for downloading the file. Information required to create the URL are:
- Snapshot date = Value of key '@id' in JSON response above OR use "latest" as shortcut which defaults to the latest available date. So if you have only 1 date, then that itself is the latest. If there are multiple dates like [20200501, 20200502, 20200503] then latest will default to 20200503.
- Request id = Name of request file (request_id) with ".bbg" extension.
- Location where you wish to store your response file.

In [ ]:
snap_date = 'latest' ## Or use the date in "@id" key above example '20200511'

download_url = snap_url+snap_date+"/distributions/"+request_id+".bbg"
output_file = request_id+".bbg"

print("GETDATA response will be downloaded using URL: "+download_url)

## Now lets call download_distribution() and pass the URL for downloading the latest snapshot.
download_resp = download_distribution(session, download_url, output_file, chunk_size=4096)
download_resp.status_code